In [ ]:
!pip install opencv-python
!pip install matplotlib

In [1]:
import cv2 as cv
import os
import numpy as np

In [2]:
DIMENSIUNE_PATRAT = 140
HEIGHT_CAREU = 14 * DIMENSIUNE_PATRAT
WIDTH_CAREU = 14 * DIMENSIUNE_PATRAT

lines_vertical = []
for i in range(0, WIDTH_CAREU+5, DIMENSIUNE_PATRAT):
    l = []
    l.append((i, 0))
    l.append((i, WIDTH_CAREU-3))
    lines_vertical.append(l)

lines_horizontal = []
for i in range(0, HEIGHT_CAREU+5, DIMENSIUNE_PATRAT):
    l = []
    l.append((0, i))
    l.append((HEIGHT_CAREU - 3, i))
    lines_horizontal.append(l)

In [3]:
folder_proiect = os.getcwd() #Tema1
folder_imagini_proprii = os.path.join(folder_proiect, 'imagini_proprii')
folder_imagini = os.path.join(folder_proiect, 'fake_test') # Tema1/antrenare, aici se va schimba in testare
continut_folder_imagini = [x for x in os.listdir(folder_imagini) if x.endswith('.txt') or x.endswith('.jpg')]

def show_image(title, image):
    image = cv.resize(image,(0,0), fx=0.25, fy=0.25)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def read_image(image_path):
    #openCv o citeste ca fiind BGR
    img = cv.imread(image_path)
    return img.copy()
    
def apply_hsv_filter(image, l_h, l_s, l_v, u_h, u_s, u_v, verbose = False):
    #BGR in BGR out
    #h -> hue
    #s -> saturation
    #v -> value
    #l -> low
    #h -> high
    copie = image.copy()
    copie = cv.cvtColor(copie, cv.COLOR_BGR2HSV)
    l = np.array([l_h, l_s, l_v])#lower bound pe cele 3 canale
    u = np.array([u_h, u_s, u_v])#upper bound pe cele 3 canale
    mask_table_hsv = cv.inRange(copie, l, u)
    
    if verbose:
        show_image('masca_hsv', mask_table_hsv)
        
    rez = cv.bitwise_and(copie, copie, mask=mask_table_hsv)
    rez = cv.cvtColor(rez, cv.COLOR_HSV2BGR)
    return rez

def _find_contour_positions(edges, verbose = False):

    if verbose:
        show_image('edge_image', edges)
        
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    return top_left, top_right, bottom_left, bottom_right

def extrage_tabla_joc(imagine, verbose = False):
    #in IMAGINE out TABLA DE JOC in BGR(Cu tot cu scrisul Mathable)
    original = imagine.copy()
    imagine = apply_hsv_filter(imagine, 14, 0, 0, 255, 255, 255)#primul parametru e bun de la 14 in sus

    if verbose:
        show_image('imagine_dupa_filtru_hsv', imagine)

    imagine = cv.cvtColor(imagine, cv.COLOR_BGR2GRAY)
    # image_m_blur = cv.medianBlur(imagine, 5)
    # image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 18)
    # image_sharpened = cv.addWeighted(image_m_blur, -0.8, image_g_blur, 1.2, 0)
    # image_sharpened = image_m_blur
    # show_image('imagine_dupa_sharpening', image_m_blur)
    
    # _, thresh = cv.threshold(image_sharpened, 40, 255, cv.THRESH_BINARY)
    # kernel = np.ones((6,6), np.uint8)
    # thresh = cv.erode(thresh, kernel)
    # show_image('image_thresholded', thresh)

    # edges = cv.Canny(thresh, 10, 255)

    top_left, top_right, bottom_left, bottom_right = _find_contour_positions(imagine, verbose)
    
    image_copy = cv.cvtColor(imagine.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    if verbose:
        show_image("detected corners",image_copy)

    width = top_right[0] - top_left[0] #18x18, each cell will have 80 pixels
    height = width
    tabla_joc = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    colutri = np.array([[0,0], [width, 0], [width, height], [0,height]], dtype='float32')
    M = cv.getPerspectiveTransform(tabla_joc, colutri)
    result = cv.warpPerspective(original, M, (width, height))
    crop_image_pixels = 85
    return result[crop_image_pixels:result.shape[0] - crop_image_pixels, 
                    crop_image_pixels:result.shape[1] - crop_image_pixels]



In [4]:
#edge detection is susceptible to noise in the image

In [5]:
#ATENTIE LA CELULA ASTA https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
def _extrage_careu_tabla(edges, imagine, verbose = False):
    #show_image('edge_image', edges)
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    squares = []
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            squeezed_contours = contours[i].squeeze()
            x_min = np.min(squeezed_contours[:, 0])
            x_max = np.max(squeezed_contours[:, 0])
            y_min = np.min(squeezed_contours[:, 1])
            y_max = np.max(squeezed_contours[:, 1])
            top_left_point = [x_min, y_min]
            top_right_point = [x_max, y_min]
            bottom_left_point = [x_min, y_max]
            bottom_right_point = [x_max, y_max]
            height = y_max - y_min
            width = x_max - x_min
            diagonal = np.sqrt(height**2 + width**2)

            if diagonal > 50:#find a good diagonal threshold
                cv.circle(imagine, tuple(top_left_point), 20, (0,0, 255), -1)
                cv.circle(imagine, tuple(top_right_point), 20, (0,0, 255), -1)
                cv.circle(imagine, tuple(bottom_left_point), 20, (0,0, 255), -1)
                cv.circle(imagine, tuple(bottom_right_point), 20, (0,0, 255), -1)
                squares.append([top_left_point, top_right_point, bottom_left_point, bottom_right_point])

    squares_np = np.array(squares)

    if verbose:
        show_image('detected corners', imagine)

    x_min = np.min(squares_np[:, :, 0])
    x_max = np.max(squares_np[:, :, 0])
    y_min = np.min(squares_np[:, :, 1])
    y_max = np.max(squares_np[:, :, 1])
    top_left_point = [x_min, y_min]
    top_right_point = [x_max, y_min]
    bottom_left_point = [x_min, y_max]
    bottom_right_point = [x_max, y_max]
    
    return (top_left_point, top_right_point, bottom_left_point, bottom_right_point)

def extrage_careu_joc(imagine, verbose = False):
    #in TABLA DE JOC out DOAR CAREUL in BGR(cred)
    original = imagine.copy()
    
    if verbose:
        show_image('imagine_tabla_joc', imagine)

    copie = imagine.copy()
    
    imagine = apply_hsv_filter(imagine, 14, 0, 0, 69, 255, 255)
    
    if verbose:
        show_image('imagine_dupa_hsv', imagine)
    
    imagine = cv.cvtColor(imagine, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(imagine, 5)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 5)
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    
    if verbose:
        show_image('image_sharpened', image_sharpened)
    
    _, thresh = cv.threshold(image_sharpened, 0, 255, cv.THRESH_BINARY)

    kernel = np.ones((10,10), np.uint8)
    thresh = cv.erode(thresh, kernel)
    
    if verbose:
        show_image('image_thresholded', thresh)

    edges = cv.Canny(thresh, 200, 400)
    #top_left, top_right, bottom_left, bottom_right = find_contour_positions(edges)
    top_left, top_right, bottom_left, bottom_right = _extrage_careu_tabla(edges, copie, verbose)
    
    image_copy = cv.cvtColor(imagine.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    if verbose:
        show_image("detected corners",image_copy)

    height = HEIGHT_CAREU
    width = WIDTH_CAREU
    
    careuri = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    destinatie_careuri = np.array([[0,0],[width, 0], [width, height], [0, height]], dtype='float32')

    M = cv.getPerspectiveTransform(careuri, destinatie_careuri)
    
    result = cv.warpPerspective(original, M, (width, height)) 

    return result

In [18]:
def cauta_pozitie_piesa_noua(imag1, imag2, verbose = False):
    #imag1 este imaginea anterioara, iar imag2 este imaginea curenta unde ne uitam dupa piese
    #in IMAG1, IMAG2 in format BGR out diferenta
    #IMAG1 si IMAG2 sunt imagini doar cu patratele de joc
    #imag2 - imag1
    global lines_horizontal
    global lines_vertical
    
    imag1 = cv.cvtColor(imag1, cv.COLOR_BGR2GRAY)
    imag2 = cv.cvtColor(imag2, cv.COLOR_BGR2GRAY)
    diferenta = cv.absdiff(imag1, imag2)
    _, diferenta = cv.threshold(diferenta, 60, 255, cv.THRESH_BINARY)
    
    if verbose:
        show_image('diferenta', diferenta)

    matrice_medie = np.zeros((14,14), dtype='float32')

    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]# + 10
            y_max = lines_vertical[j + 1][1][0]# - 15
            x_min = lines_horizontal[i][0][1]# + 10
            x_max = lines_horizontal[i + 1][1][1]# - 10
            patch = diferenta[x_min:x_max, y_min:y_max].copy()
            medie = np.mean(patch)
            matrice_medie[i,j] = medie

    index_max = np.argmax(matrice_medie)
    i_max, j_max = np.unravel_index(index_max, matrice_medie.shape)
    return int(i_max), int(j_max)#numaratoare de la 0


In [19]:
def adnotari_prima_cerinta(id_imagine, verbose = False):
    adnotari = []
    base_imagine = read_image(os.path.join(folder_imagini_proprii, 'tabla.jpg'))
    base_tabla = extrage_tabla_joc(base_imagine)
    base_careu = extrage_careu_joc(base_tabla)
    for i in range(1, 51):       
        imagine = read_image(os.path.join(folder_imagini, f'{id_imagine}_{i//10}{i%10}.jpg'))
        imagine_tabla = extrage_tabla_joc(imagine, verbose)
        imagine_careu = extrage_careu_joc(imagine_tabla, verbose)
        i, j = cauta_pozitie_piesa_noua(base_careu, imagine_careu, verbose)        
        base_careu = imagine_careu
        adnotari.append((i, j))
    return adnotari

def scrie_adnotari(id_imagine, adnotari):
    folder_solutii = os.path.join(folder_proiect, '334_Florete_FabianAndrei')
    alfabet = [chr(x) for x in range(ord('A'), ord('a') + 15)]
    for i in range(1, 51):
        f = open(os.path.join(folder_solutii, f'{id_imagine}_{i//10}{i%10}.txt'), 'w')
        f.write(f'{adnotari[i - 1][0] + 1}{alfabet[adnotari[i - 1][1]]} 0')
        f.close()

In [20]:
a = adnotari_prima_cerinta(1)
scrie_adnotari(1, a)